In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def open_boundary(h_symbolic, kx_sym, Nx, Nbands):
    Lx_nn_pos = sp.integrate(h_symbolic * sp.exp(sp.I * kx_sym * (-1.0)), (kx_sym, -sp.pi, sp.pi))/ (2 * sp.pi)
    Lx_nn_neg = sp.integrate(h_symbolic * sp.exp(sp.I * kx_sym), (kx_sym, -sp.pi, sp.pi))/ (2 * sp.pi)
    Lx_nnn_pos = sp.integrate(h_symbolic * sp.exp(sp.I * 2 * kx_sym * (-1.0)), (kx_sym, -sp.pi, sp.pi))/ (2 * sp.pi)
    Lx_nnn_neg = sp.integrate(h_symbolic * sp.exp(sp.I * 2 * kx_sym), (kx_sym, -sp.pi, sp.pi))/ (2 * sp.pi)

    Lx_nn_pos = Lx_nn_pos.rewrite(sp.cos).simplify()
    Lx_nn_neg = Lx_nn_neg.rewrite(sp.cos).simplify()
    Lx_nnn_pos = Lx_nnn_pos.rewrite(sp.cos).simplify()
    Lx_nnn_neg = Lx_nnn_neg.rewrite(sp.cos).simplify()

    H_diag = h_symbolic - (Lx_nn_pos * sp.exp(sp.I * kx_sym) + Lx_nn_neg * sp.exp(-sp.I * kx_sym) + Lx_nnn_pos * sp.exp(sp.I * 2 * kx_sym) + Lx_nnn_neg * sp.exp(-sp.I * 2 * kx_sym))
    H_diag = H_diag.rewrite(sp.cos).simplify()
    H_diag = sp.nsimplify(H_diag, tolerance = 1e-8)

    h = sp.zeros(Nx*Nbands, Nx*Nbands)

    for i in range(Nx):
        h[i*Nbands:(i+1)*Nbands, i*Nbands:(i+1)*Nbands] = H_diag[:,:]

        if i > 0: 
            h[(i-1)*Nbands:i*Nbands, i*Nbands:(i+1)*Nbands] = Lx_nn_pos[:,:]

        if i > 1:
            h[(i-2)*Nbands:(i-1)*Nbands, i*Nbands:(i+1)*Nbands] = Lx_nnn_pos[:,:]

        if i < Nx - 1:
            h[(i+1)*Nbands:(i+2)*Nbands, i*Nbands:(i+1)*Nbands] = Lx_nn_neg[:,:]

        if i < Nx - 2:
            h[(i+2)*Nbands:(i+3)*Nbands, i*Nbands:(i+1)*Nbands] = Lx_nnn_neg[:,:]
    
    slab_h = np.array(h).astype(np.complex128)

    return slab_h

In [ ]:
kx_sym = sp.symbols('k_x', real = True)
alpha_sym = sp.symbols('alpha', real = True, positive = True)

In [ ]:
s0 = sp.eye(2)
sx = sp.Matrix([[0, 1], [1, 0]])
sy = sp.Matrix([[0, -sp.I], [sp.I, 0]])
sz = sp.Matrix([[1, 0], [0, -1]])

PauliList = [s0, sx, sy, sz] 

In [ ]:
h_1d = (sp.sin(kx_sym) + alpha_sym * sp.sin(2*kx_sym)) * sx
h_1d += (sp.cos(kx_sym) + 2 * sp.cos(2*kx_sym)) * sz
h_1d

In [ ]:
Nx = 100
Nbands = 2

params = {}
params["Nx"] = Nx
params["Nbands"] = Nbands

In [ ]:
H_fixparam = h_1d.subs({alpha_sym : 2.})
hfunc = sp.lambdify((kx_sym), H_fixparam, modules = "numpy")

In [ ]:
Kxs = np.linspace(0, 2*np.pi, Nx, endpoint = False)

In [ ]:
eigenvalues = np.zeros((Nx, Nbands))
eigenvectors = np.zeros((Nx, Nbands, Nbands), dtype = np.complex128)

for i, kx in enumerate(Kxs):
    values, vectors = np.linalg.eigh(hfunc(kx))
    ind = np.argsort(values)
    eigenvalues[i,:] = values[ind]
    eigenvectors[i,:,:] = vectors[:, ind]

In [ ]:
Nx2 = int(Nx/2)
halfBZ_vecs = eigenvectors[:Nx2,:,0]

W = 1 
for i in range(Nx2):
    ol = np.dot(halfBZ_vecs[(i+1)%Nx2,:], halfBZ_vecs[i,:])
    W *= ol / np.abs(ol)
print(W)
print(np.angle(W)/(2*np.pi))

In [ ]:
slab_h = open_boundary(H_fixparam, kx_sym, Nx, Nbands)

In [ ]:
eigenvalues, eigenstates = np.linalg.eigh(slab_h)
ind = np.argsort(eigenvalues)
eigenvalues = eigenvalues[ind]
eigenstates = eigenstates[:, ind]

In [ ]:
plot_data = eigenvalues

fig = plt.figure(figsize = (4,3))
plt.scatter(np.arange(len(plot_data)), plot_data, c = "black", s = 10)
plt.xlabel("Eigenvalue index", fontsize = 16)
plt.ylabel(r"$E$", fontsize = 16, labelpad = -4)
plt.yticks([-3,0,3], ["-3", "0", "3"], fontsize = 14)
plt.xticks([0,100,200], ["0", "100", "200"], fontsize = 14)
plt.show()

In [ ]:
mask = abs(eigenvalues) < 1e-0 * 1.03
plot_data = eigenvalues[mask]
plot_x = np.arange(len(eigenvalues))[mask]

fig = plt.figure(figsize = (1.5,1.5))
plt.scatter(plot_x, plot_data, c = "black", s = 10, alpha = 1)
plt.yticks(fontsize = 14)
plt.xticks(fontsize = 14)
plt.show()